In [ ]:
import numpy as np
import pandas as pd
import torch
import sys, os, copy
from tqdm.auto import tqdm, trange
from omegaconf import OmegaConf
import matplotlib.pyplot as plt
import warnings, time
import argparse, json
warnings.simplefilter("ignore")
# torch.set_printoptions(precision=3, sci_mode=False)

from datetime import datetime

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.subplots as sp
plotly_layout = dict(margin=dict(l=20, r=20, t=20, b=20))

import pybullet as p

from envs import get_env

template = copy.deepcopy(pio.templates['simple_white'])

template.layout.xaxis.showgrid = True
template.layout.xaxis.title = 'Time (s)'
template.layout.xaxis.title.standoff = 0
template.layout.yaxis.showgrid = True
template.layout.yaxis.title = 'Collision Distance (m)'

template.layout.font.family = 'Calibri'
template.layout.font.size = 30
template.layout.legend.xanchor = 'right'
template.layout.legend.yanchor = 'top'
template.layout.legend.x = 1.25
template.layout.legend.y = 0.99
template.layout.legend.bgcolor = 'rgba(0,0,0,0)'

In [ ]:
env_cfg = OmegaConf.load("configs/envs/env_config_labpanda.yml")
env = get_env(env_cfg, GUI=True)

speed_slider = p.addUserDebugParameter(f'_Speed',rangeMin=0.01, rangeMax=3, startValue=1)

vStart = 0
bStart = p.addUserDebugParameter("_Start", 1,0,vStart)

vReset = 0
bReset = p.addUserDebugParameter("_Reset", 1,0,vReset)

vPause = 0
bPause = p.addUserDebugParameter("_Pause", 1,0,vPause)

In [ ]:
demo_csv = pd.read_csv('planning/data/labpanda/log_data_2409031437.csv', header=0)
T = demo_csv['time'].values
T = T - T[0]
min_distance = torch.tensor(demo_csv['min_distance'].values, dtype=torch.float)
qs = demo_csv['current_joint'].values
qs = torch.tensor([json.loads(q) for q in qs], dtype=torch.float)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=T, y=min_distance, mode='lines', line_width=3, line_color='black', name='PairwiseNet'))
fig.update_yaxes(range=[-0.07, 0.5])
fig.update_layout(**plotly_layout, width=1280, height=480, template=template)
fig.show()

In [ ]:
traj_idx = 0
env.env_bullet.reset2TargetPositions(qs[traj_idx])

ongoing_flag = False

t_old = time.time()
current_t = 0

while True:
    
    if int(p.readUserDebugParameter(bStart)) != vStart:
        vStart = int(p.readUserDebugParameter(bStart))
        print('Simulation Start')
        ongoing_flag = True
        
    if int(p.readUserDebugParameter(bReset)) != vReset:
        vReset = int(p.readUserDebugParameter(bReset))
        print('Simulation Reset')
        ongoing_flag = False
        env.env_bullet.reset2TargetPositions(qs[0])
        t_old = time.time()
        current_t = 0
        
    if int(p.readUserDebugParameter(bPause)) != vPause:
        vPause = int(p.readUserDebugParameter(bPause))
        print('Simulation Pause')
        ongoing_flag = False
    
    speed = p.readUserDebugParameter(speed_slider)    
    
    if ongoing_flag:
        current_t += (time.time() - t_old) * speed    
    t_old = time.time()

    traj_idx = abs(T-current_t).argmin()
        
    env.env_bullet.reset2TargetPositions(qs[int(traj_idx)])
    
    dist, points = env.env_bullet.check_collision(qs[int(traj_idx)], return_points=True)
    # print(f'Min. dist. {dist:.3f}m')
    p.addUserDebugLine(points[0], points[1], [1, 0, 0], 5, 0.1)
    